# Sample notebook: Analyses

---
This notebook is based on the code from "https://github.com/GVS-Lab/germinal_center/" by Daniel Paysan and Saradha Venkatachalapathy (2023) and serves as an illustrating example of how to analyze biopsy images using the chrometric and protein features obtained as output of the preceding data processing.

Briefly, this notebook provides functionality to
1. Identify cell types based on protein features
2. Filter out uninformative observations and features
3. Perform statistical screen to identify chrometric cell type markers
4. Compute cell-cell distances

---


---

## Setting up the environment

As a first step, we again load a number of external software packages, that we will use.



In [281]:
# import libraries
import sys
from pathlib import Path
from glob import glob
import pandas as pd
import numpy as np
from collections import Counter
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
from skimage.measure import regionprops
import cv2
from statannotations.Annotator import Annotator
import warnings

warnings.filterwarnings("ignore")
seed = 1234
plt.rcParams["figure.dpi"] = 300

%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

We also load a number of function defined within this repository. Please refer to the code of these for a better understanding of what they do.

In [ ]:
sys.path.append("../..")

from src.utils.cell_type_detection import *
from src.utils.data_viz import *
from src.utils.data_processing import clean_data, remove_correlated_features
from src.utils.discrimination import *

---

## Read in the preprocessed data


We second read in the data that was generated by the preceding image processing, namely the segmentation and feature profiling tasks.

In [139]:
nuc_features = pd.read_csv("/path/to/nuc_features.csv", index_col=0)



spatial_cord = pd.read_csv("/path/to/spatial_coordiates.csv"), index_col=0)
spatial_cord.index = spatial_cord["nuc_id"]

lamin_levels = pd.read_csv("/path/to/lamin_levels.csv", index_col=0)
cd3_levels = pd.read_csv("/path/to/cd3_levels.csv", index_col=0)

<IPython.core.display.Javascript object>

While this is not required, we recommend renaming the chrometric features according to their updated description, this is achieved by running the code below.

In [ ]:
nuc_feature_description = pd.read_csv(
    "https://github.com/GVS-Lab/chrometrics/blob/main/chrometric_feature_description.csv", index_col=0
)
feature_name_dict = dict(
    zip(
        list(nuc_feature_description.loc[:, "feature"]),
        list(nuc_feature_description.loc[:, "long_name"]),
    )
)
nuc_features = nuc_features.rename(columns=feature_name_dict)

Note that the linked ``.csv`` file also contains a description of the features, which might be helpful to better understand what these features are.

---

## Identify cell type labels

To identify the cell type labels, we will use the expression of marker proteins that were measured. In this example these are only the CD3 labels but the procedure shown below can be similarly run if there are many more marker stains available and profiled using the preceding imaging processing described in the feature generation notebook.

To identify if a cell is stained positively for a given marker, we look at the average intensity of the corresponding protein within the identified cellular mask. Assuming sufficient specificity of the staining, we should observe a bimodal distribution of that quantity when looking at the mean expression for all cells in a given image. Cells that are positive for the marker will contribute to the higher mode and those that are negative to the lower mode. We thus, identify cells that are positive for a marker by fitting a 2-component Gaussian mixture model for the average cellular intensities of the marker protein and label cells as positive that are assigned to the component with the larger mode and others as negative.

In [ ]:
(_, fovs) = pd.factorize(cd4_levels["image"].astype("category"))
img_names = fovs.categories
cd3_positive_cells = get_positive_cells_batch(cd3_levels, img_names)

Note that this is done individually for each image, as the intensity distribution of the markers might vary between images.

We store the identified cell type labels as a new feature called ``cd3_status`` as part of our nuclear features.

In [ ]:
nuc_features["cd3_status"] = "negative"
nuc_features.loc[
    nuc_features.nuc_id.isin(cd3_positive_cells), "cd3_status"
] = "positive"

---

## Chrometric feature preprocessing

We next preprocess the chrometric features for consecutive analyses. This will mostly include filtering out constant features and observations and those that contain missing values. Additionally, we remove a number of features that provide positional information and should not be considered in our analyses.

In [ ]:
metacolumns = [
    "label_id",
    "weighted_centroid_y",
    "weighted_centroid_x",
    "centroid_y",
    "centroid_x",
    "bbox-0",
    "bbox-1",
    "bbox-2",
    "bbox-3",
    "nuc_id",
    "image",
    "orientation",
    "spat_centroid_y",
    "spat_centroid_x",
    "cd3_status"
]
# drop_columns = set(metacolumns).union(set(list(gc_nuc_features.filter(regex="-").columns)))
drop_columns = metacolumns
cleaned_nuc_features = clean_data(
    nuc_features, drop_columns=list(drop_columns), index_col="nuc_id"
)

cleaned_nuc_cd3_labels = nuc_features.loc[nuc_features.nuc_id.isin(cleaned_nuc_features.index), "cd3_status"]

---

## Identify chrometric cell type markers

We now identify chrometric features that are significantly different between CD3+ and CD3- negative cells. Note that this procedure can be similarly run for any other labels such as e.g. health and disease if these are available.

In particular, we run a test-based screen that for each chrometric feature uses a statistical test defined as the ``test`` parameter (here the Welch t-test) to identify differences in the distribution or here means of the considered chrometric features in cells that are positive or negative in our example for CD3.

The results are filtered to remove all entries that are not significant on a 5% confidence level after adjusting for multiple testing using FDR correction.

In [ ]:
celltype_marker_results = find_markers(
    data=cleaned_nuc_features, labels=cleaned_nuc_cd3_labels, test="welch"
)
tcell_marker_results = celltype_marker_results.loc[
    celltype_marker_results.label == "positive"
]
tcell_marker_results = tcell_marker_results.loc[
    tcell_marker_results.adjusted_pval < 0.05
]
tcell_marker_results = tcell_marker_results.sort_values("abs_delta_fc", ascending=False)
tcell_marker_results

---

## Visualizing the results

To visualize the differences, one could for instance plot the distributions of e.g. the top feature when sorting them based on the absolute log-fold change using violinplots.

In [ ]:
fig, ax = plt.subplots(figsize=[6,4])
ax = sns.violinplot(data = nuc_features.loc[nuc_features.nuc_id.isin(cleaned_nuc_features.index)], x="cd3_status", 
                    y=tcell_marker_results.iloc[0,1])
plt.show()

---

## Measure cellular distances

Finally, we compute the pair-wise euclidean distance matrices for all cells in each image, which can be used for later analyses. Note that depending on the number of cells this matrix can get extremely large.

In [ ]:
image_ids, cell_dists = get_distances_to_tcells(
    dataset=nuc_features,
    id_col="nuc_id",
    range_norm=True,
)

The output of the function will be a list of the image IDs and a list with the corresponding cellular distance matrices. The parameter ``range_norm`` marks if the distances should be range-normalized to 0 and 1 within each distance matrix.

The row and column names mark the ``nuc_id`` of the cells whose distance is given at the respective entry, which also serves as the index for e.g. the nuc_features data frame and thus enables the mapping of distances of cells to their respective chrometric and proteomic profiles.

As an example, we here plot the distance matrix of the first image.

In [ ]:
print(image_ids[0])
cell_dists[0]